In [1]:
%cd ~/Dropbox/programs/critic
import numpy as np
import jax.numpy as jnp
import jax
import pandas as pd
import plotly.express as px
import matplotlib.pyplot as plt
import seaborn as sns
import rho_plus as rp
import wat

is_dark = False
theme, cs = rp.mpl_setup(is_dark)

/home/nicholas/Dropbox/programs/critic


In [2]:
from transformers import AutoConfig, FlaxAutoModelForCausalLM, AutoTokenizer

model_name = 'distilbert/distilgpt2'
model = FlaxAutoModelForCausalLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

/home/nicholas/anaconda3/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
CUDA backend failed to initialize: Unable to use CUDA because of the following issues with CUDA components:
Unable to load cuDNN. Is it installed?
Traceback (most recent call last):
  File "/home/nicholas/anaconda3/lib/python3.11/site-packages/jax/_src/xla_bridge.py", line 322, in _version_check
    version = get_version()
              ^^^^^^^^^^^^^
RuntimeError: cuDNN not found.
 (Set TF_CPP_MIN_LOG_LEVEL=0 and rerun for more info.)
/home/nicholas/anaconda3/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `forc

In [3]:
option_strs = ['tere', 'tear', 'tree', 'terr', 'ere', 'tee', 'terse', 'there', 'sere', 'tire', 'tare', 'tern', 'tore', 'term', 'mere', 'here', 'were']
options = jnp.array([x[0] for x in tokenizer.batch_encode_plus([' ' + x for x in option_strs])['input_ids']])
options

Array([  256, 11626,  5509,  8812,   304, 30479,  1059,   612,   384,
       15867,   256,  1059, 26197,  3381,  5019,   994,   547],      dtype=int32)

In [4]:
inputs = tokenizer("That's neither here nor", return_tensors="np")
outputs = model(**inputs)

# retrieve logts for next token
next_token_logits = outputs.logits[:, -1][0]
next_token_logits

Array([-59.337803, -57.187347, -63.34384 , ..., -66.71974 , -64.19488 ,
       -60.27142 ], dtype=float32)

In [5]:
probs = jax.nn.softmax(next_token_logits)

option_probs = jax.nn.softmax(probs[options])

values, indices = jax.lax.top_k(option_probs, k=16)
tokens = tokenizer.batch_decode(options[indices])
results = pd.Series(values, index=tokens)
print(results.round(3))

values, indices = jax.lax.top_k(probs, k=16)
tokens = tokenizer.batch_decode(indices)
results = pd.Series(values, index=tokens)
print(results.round(3))

there    0.090
here     0.063
were     0.056
se       0.056
t        0.056
t        0.056
e        0.056
mere     0.056
ter      0.056
ter      0.056
term     0.056
tear     0.056
tree     0.056
terr     0.056
tee      0.056
tire     0.056
dtype: float32
there        0.470
in           0.150
here         0.103
anywhere     0.061
at           0.017
the          0.014
now          0.014
on           0.011
elsewhere    0.010
yet          0.010
ever         0.006
abroad       0.005
any          0.005
I            0.004
a            0.003
near         0.003
dtype: float32
